In [4]:
  # define rooms and items

bathtub = {
    "name": "bathtub",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

door_d = {
    "name": "door d",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}

sink = {
    "name": "sink",
    "type": "furniture",
}

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

swing = {
    "name": "swing",
    "type": "furniture",
}

pole_dancing_bar = {
    "name": "pole dancing bar",
    "type": "furniture",
}

flower_pot = {
    "name": "flower pot",
    "type": "furniture",
}

bath_room = {
    "name": "bath room",
    "type": "room",
}

captain_room = {
    "name": "captain room",
    "type": "room",
}

toy_room = {
    "name": "toy room",
    "type": "room",
}

hallway = {
    "name": "hallway",
    "type": "room",
}

outside = {
  "name": "outside"
}

all_rooms = [bath_room, captain_room, toy_room, hallway, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    "bath room": [sink, bathtub, door_a],
    "sink": [key_a],
    "door a": [bath_room, captain_room],
    "captain room" : [queen_bed,door_a,door_b,door_c],
    "queen bed": [key_b],
    "door b" : [captain_room,toy_room],
    "toy room" : [swing, pole_dancing_bar, door_b],
    "pole dancing bar" : [key_c],
    "swing" : [key_d],
    "door c" : [captain_room,hallway],
    "hallway" : [flower_pot,door_c,door_d],
    "door d" : [outside],
    "outside": [door_d],
}

# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": bath_room,
    "keys_collected": [],
    "target_room": outside
}

In [6]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("""\n
                  __|__|__\n
                __|_o_o_o_|__\n
           ____/___________|__\\____\n
           \\                   o o  /\n
    ~~~~~~~~\\______________________/~~~~~~~~\n
       ~~~~~~                        ~~~~~~
\n
It's been a hard night in the Winter Pride Mediterranean Gay Cruise Festival.\n
While you were having a great time flirting with the cruise captain, you began to feel\n
dizzy after having a lemon vodka drink he invited you to. He told you to get to his quarters but you lost consciousness.\n
Now you wake up in bathtub you are not aquainted to, in a bathroom it's not yours.\n
 
""")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped from the clutches of the evil captain!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")

    if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

In [8]:
game_state = INIT_GAME_STATE.copy()

start_game()



                  __|__|__

                __|_o_o_o_|__

           ____/___________|__\____

           \                   o o  /

    ~~~~~~~~\______________________/~~~~~~~~

       ~~~~~~                        ~~~~~~


It's been a hard night in the Winter Pride Mediterranean Gay Cruise Festival.

While you were having a great time flirting with the cruise captain, you began to feel

dizzy after having a lemon vodka drink he invited you to. He told you to get to his quarters but you lost consciousness.

Now you wake up in bathtub you are not aquainted to, in a bathroom it's not yours.

 

You are now in bath room


What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? sink


You examine sink. You find key for door a.
You are now in bath room


What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? examine


The item you requested is not found in the current room.
You are now in bath room


...

You are now in hallway


What would you like to do? Type 'explore' or 'examine'? examine
What would you like to examine? door d


You examine door d. You unlock it with a key you have.


Do you want to go to the next room? Enter 'yes' or 'no' yes


Congrats! You escaped from the clutches of the evil captain!



















































